In [1]:
from game import Game
import threading
import time
import random

Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [2]:
game_instance = Game()

In [3]:
def start_game():
    # start the game
    if not game_instance.get_isrunning():
        game_instance.start()

In [4]:
def transform_nodes(old_nodes):
    # print('-------------',type(old_nodes))
    new_nodes = []
    for node in old_nodes:
        new_node = {
            "id": node.id,
            "label": str(node.id),
            "layer": 1,
            "x": node.x,
            "y": node.y,
            "color": {"background": str(node.color)},
            "value": 30,
        }
        new_nodes.append(new_node)
        # print(new_node)
    return new_nodes

def transform_edges(edges):
    transformed_edges = []
    for idx, edge in enumerate(edges, start=1):
        transformed_edges.append({
            "id": idx,
            "from": edge[0],
            "to": edge[1]
        })
    return transformed_edges

In [5]:
game_thread = threading.Thread(target=start_game)
game_thread.start()
time.sleep(12)

Period from 0.000 to 0.501
Adversary: Start SCAN_HOST at 0.0s.
Adversary: Processed SCAN_HOST at 5.0s.
Adversary: Start ENUM_HOST at 5.0s.
Adversary: Processed ENUM_HOST at 10.0s.
Adversary: Start SCAN_PORT at 10.0s.
Adversary: Processed SCAN_PORT at 35.0s.
Adversary: Start SCAN_HOST at 130.5s.
Adversary: Processed SCAN_HOST at 135.5s.
Adversary: Start ENUM_HOST at 135.5s.
Adversary: Processed ENUM_HOST at 140.5s.
Adversary: Start SCAN_PORT at 140.5s.
Period from 0.501 to 1.012
Adversary: Processed SCAN_PORT at 165.5s.
Period from 1.012 to 1.518
Adversary: Start SCAN_HOST at 331.0s.
Adversary: Processed SCAN_HOST at 336.0s.
Adversary: Start ENUM_HOST at 336.0s.
Adversary: Processed ENUM_HOST at 341.0s.
Adversary: Start SCAN_PORT at 341.0s.
Adversary: Processed SCAN_PORT at 366.0s.
Period from 1.518 to 2.028
Adversary: Start SCAN_HOST at 541.3s.
Adversary: Processed SCAN_HOST at 546.3s.
Adversary: Start ENUM_HOST at 546.3s.
Adversary: Processed ENUM_HOST at 551.3s.
Adversary: Start SCAN

In [19]:
compromised_hosts = game_instance.adversary.get_compromised_hosts()

In [20]:
compromised_hosts

[0, 8, 9, 5, 19, 6]

In [21]:
visible_network = game_instance.time_network.get_hacker_visible_graph()

In [25]:
uncompromised_hosts = []
network = game_instance.time_network
# Add every uncompromised host that is reachable and is not an exposed or compromised host
for c_host in compromised_hosts:
    uncompromised_hosts = uncompromised_hosts + [
        neighbor
        for neighbor in network.graph.neighbors(c_host)
        if neighbor not in compromised_hosts and neighbor not in network.exposed_endpoints and
           len(network.get_path_from_exposed(neighbor, graph=visible_network)[0]) > 0
    ]

uncompromised_hosts = uncompromised_hosts + [
            ex_node
            for ex_node in network.exposed_endpoints
            if ex_node not in compromised_hosts
        ]

In [26]:
uncompromised_hosts

[17, 7, 17, 1, 2, 3, 4]

In [27]:
stop_attack = game_instance.adversary.get_stop_attack()

In [28]:
discovered_hosts = [n for n in uncompromised_hosts if n not in stop_attack]

In [29]:
discovered_hosts

[17, 7, 17, 1, 2, 3, 4]

In [30]:
compromised_hosts + uncompromised_hosts

[0, 8, 9, 5, 19, 6, 17, 7, 17, 1, 2, 3, 4]

In [31]:
adversary = game_instance.adversary
adversary.set_curr_process('SCAN_HOST')
network = game_instance.time_network
adversary.set_host_stack(network.sort_by_distance_from_exposed_and_pivot_host(
            adversary.get_host_stack(),
            adversary.get_compromised_hosts(),
            pivot_host_id=adversary.get_pivot_host_id()
        ))

In [35]:
adversary.get_host_stack()

[3, 4]

In [36]:
adversary.get_curr_host().port_scan()

[4410, 49998, 19680, 31376, 28235, 48306, 3762, 32627, 44945, 32164, 45230]

In [37]:
adversary.set_curr_vulns(adversary.get_curr_host().get_vulns(adversary.get_curr_ports()))
adversary.get_curr_vulns()